# Housing Prices

### A project was created to assimilate knowledge of an eleven-week machine learning course by Andrew NG. Additionally, the problem being solved allowed me to broaden my knowledge of feature engineering and new models.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # Display full output

## Data understanding

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
df=pd.read_csv('data/train.csv')
df.head()
df.shape

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


(1460, 81)

### Group columns by datatype

In [3]:
# Nominal columns
columnsNom = ''.join('''MSSubClass, MSZoning, Alley, LandContour, 
LotConfig, Neighborhood, Condition1, Condition2, 
BldgType, HouseStyle, RoofStyle, RoofMatl, Exterior1st, Exterior2nd, 
MasVnrType, Foundation, Heating, Electrical, GarageType, MiscFeature, 
SaleType, SaleCondition, Street'''.split('\n')).split(', ')

# Ordinal columns
columnsOrd = ''.join('''LotShape, Utilities, LandSlope, ExterQual, ExterCond, BsmtQual, BsmtCond,
 BsmtExposure, BsmtFinType1, BsmtFinType2, HeatingQC, CentralAir, KitchenQual,
 Functional, FireplaceQu, GarageFinish, GarageQual, GarageCond, PavedDrive, PoolQC,
 Fence'''.split('\n')).split(', ')

# Others are numeric columns

## Data preprocessing

### Feature engineering I

New feastures ideas:
- Encoding Nominal columns
- Convert Ordinal to Numeric
- Feature interactions
- Others like yes_no, max_min, with 5% floor, etc.
- Log
- Rank
- Polynomials
- Bucketing
- Splines

In [4]:
# One hot encoding for Nominal data
newColumnsNom=[]
oldColumnList=df.columns
for col in columnsNom: 
    pref='Nominal_'+col
    df = pd.concat([df, pd.get_dummies(df[col], prefix=pref)],axis=1)
    df.drop([col],axis=1, inplace=True) # Drop the original column
newColumnsNom=[x for x in df.columns if x not in oldColumnList]

In [5]:
# Convert Ordinal data to Numeric data
columnsOrdVal=[]
for col in columnsOrd: # Number the values
    dict={}
    for i in range(len(df[col].unique())):
        dict[df[col].unique()[i]]=len(df[col].unique())-i
    columnsOrdVal.append(dict)

for i in range(len(columnsOrd)): # Replace values with numbers
    df[columnsOrd[i]]=df[columnsOrd[i]].replace(columnsOrdVal[i])

### Handling NaNs

In [6]:
for col in df.columns:
    if df[col].isna().sum()!=0:
        print(col+': ')
        df[col].isna().sum()

LotFrontage: 


259

MasVnrArea: 


8

GarageYrBlt: 


81

In [7]:
df[df['GarageYrBlt'].isna()]['GarageArea'].unique()
# GarageYrBlt is Nan <=> GarageArea is 0 so there is no Garage

array([0], dtype=int64)

In [8]:
data=df[df['LotFrontage'].isna()]
for col in data.columns:
    print(data[["LotFrontage",col]].corr())
# MasVnrArea is Nan => there is no Masonry veneer

             LotFrontage   Id
LotFrontage          NaN  NaN
Id                   NaN  1.0
             LotFrontage  LotFrontage
LotFrontage          NaN          NaN
LotFrontage          NaN          NaN
             LotFrontage  LotArea
LotFrontage          NaN      NaN
LotArea              NaN      1.0
             LotFrontage  LotShape
LotFrontage          NaN       NaN
LotShape             NaN       1.0
             LotFrontage  Utilities
LotFrontage          NaN        NaN
Utilities            NaN        1.0
             LotFrontage  LandSlope
LotFrontage          NaN        NaN
LandSlope            NaN        1.0
             LotFrontage  OverallQual
LotFrontage          NaN          NaN
OverallQual          NaN          1.0
             LotFrontage  OverallCond
LotFrontage          NaN          NaN
OverallCond          NaN          1.0
             LotFrontage  YearBuilt
LotFrontage          NaN        NaN
YearBuilt            NaN        1.0
              LotFrontage  YearRemodA

Nominal_LandContour_Bnk          NaN                      1.0
                         LotFrontage  Nominal_LandContour_HLS
LotFrontage                      NaN                      NaN
Nominal_LandContour_HLS          NaN                      1.0
                         LotFrontage  Nominal_LandContour_Low
LotFrontage                      NaN                      NaN
Nominal_LandContour_Low          NaN                      1.0
                         LotFrontage  Nominal_LandContour_Lvl
LotFrontage                      NaN                      NaN
Nominal_LandContour_Lvl          NaN                      1.0
                          LotFrontage  Nominal_LotConfig_Corner
LotFrontage                       NaN                       NaN
Nominal_LotConfig_Corner          NaN                       1.0
                           LotFrontage  Nominal_LotConfig_CulDSac
LotFrontage                        NaN                        NaN
Nominal_LotConfig_CulDSac          NaN                  

Nominal_HouseStyle_1.5Unf          NaN                        1.0
                           LotFrontage  Nominal_HouseStyle_1Story
LotFrontage                        NaN                        NaN
Nominal_HouseStyle_1Story          NaN                        1.0
                           LotFrontage  Nominal_HouseStyle_2.5Fin
LotFrontage                        NaN                        NaN
Nominal_HouseStyle_2.5Fin          NaN                        NaN
                           LotFrontage  Nominal_HouseStyle_2.5Unf
LotFrontage                        NaN                        NaN
Nominal_HouseStyle_2.5Unf          NaN                        1.0
                           LotFrontage  Nominal_HouseStyle_2Story
LotFrontage                        NaN                        NaN
Nominal_HouseStyle_2Story          NaN                        1.0
                           LotFrontage  Nominal_HouseStyle_SFoyer
LotFrontage                        NaN                        NaN
Nominal_Ho

                             LotFrontage  Nominal_Exterior2nd_HdBoard
LotFrontage                          NaN                          NaN
Nominal_Exterior2nd_HdBoard          NaN                          1.0
                             LotFrontage  Nominal_Exterior2nd_ImStucc
LotFrontage                          NaN                          NaN
Nominal_Exterior2nd_ImStucc          NaN                          1.0
                             LotFrontage  Nominal_Exterior2nd_MetalSd
LotFrontage                          NaN                          NaN
Nominal_Exterior2nd_MetalSd          NaN                          1.0
                           LotFrontage  Nominal_Exterior2nd_Other
LotFrontage                        NaN                        NaN
Nominal_Exterior2nd_Other          NaN                        NaN
                             LotFrontage  Nominal_Exterior2nd_Plywood
LotFrontage                          NaN                          NaN
Nominal_Exterior2nd_Plywood     

Nominal_SaleType_ConLI          NaN                     1.0
                        LotFrontage  Nominal_SaleType_ConLw
LotFrontage                     NaN                     NaN
Nominal_SaleType_ConLw          NaN                     NaN
                      LotFrontage  Nominal_SaleType_New
LotFrontage                   NaN                   NaN
Nominal_SaleType_New          NaN                   1.0
                      LotFrontage  Nominal_SaleType_Oth
LotFrontage                   NaN                   NaN
Nominal_SaleType_Oth          NaN                   NaN
                     LotFrontage  Nominal_SaleType_WD
LotFrontage                  NaN                  NaN
Nominal_SaleType_WD          NaN                  1.0
                               LotFrontage  Nominal_SaleCondition_Abnorml
LotFrontage                            NaN                            NaN
Nominal_SaleCondition_Abnorml          NaN                            1.0
                               LotFronta

In [9]:
def get_garage_built(values):
    if values[0]!=-1:
        return values[0]
    else:
        return values[1]

df['MasVnrArea']=df['MasVnrArea'].fillna(0)
df['LotFrontage']=df['LotFrontage'].fillna(0)
df['GarageYrBlt']=df['GarageYrBlt'].fillna(-1)
df['GarageYrBlt']=df[['GarageYrBlt','YearBuilt']].apply(get_garage_built,axis=1)

### Data visualizing

In [10]:
%matplotlib notebook
from itertools import combinations
import matplotlib.pyplot as plt

fig, axes = plt.subplots(len(df.columns)//3, 3, figsize=(12, 48))

i = 0
for triaxis in axes:
    for axis in triaxis:
        df.hist(column = df.columns[i], bins = 100, ax=axis)
        i = i+1

### Feature engineering II

In [11]:
from scipy import stats
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

y=df[['SalePrice']]
X=df.drop(['SalePrice','Id'], axis=1)

poly_cols=[]
binning_cols=[]
interaction_cols=[]
rank_cols=[]
log_cols=[]

# Polynomial features        
print('Polynomial features: ')
for col in X.columns:
    if col not in newColumnsNom:
        new_col='poly_'+col
        X[new_col]=X[col]*X[col]
        poly_cols.append(new_col)
        print(new_col)

# #Polynomial-Interaction features
# print('###Polynomial-Interaction features: ')
# for i in range(len(X.columns)):
#     for j in range(i+1,len(X.columns)):
#         try:
#             col1=X.columns[i]
#             col2=X.columns[j]
#             new_col=col1+'_'+col2
#             pvalue=stats.pearsonr(X[col1], X[col2])[1]
#             if pvalue <= 0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001 and col1!=col2:
#                 print(col1,' and ',col2,' p-value is: ',pvalue)
#                 new_col1='inter_poly_'+col1+'_'+col2+'_1'
#                 new_col2='inter_poly_'+col1+'_'+col2+'_2'
#                 new_col3='inter_poly_'+col1+'_'+col2+'_3'
#                 new_col4='inter_poly_'+col1+'_'+col2+'_4'
#                 new_col5='inter_poly_'+col1+'_'+col2+'_5'
#                 poly = PolynomialFeatures(2)
#                 X[[new_col1,new_col2,new_col3,new_col4,new_col5]]=pd.DataFrame(poly.fit_transform(X[[col1,col2]]))[[1,2,3,4,5]]
#                 print(new_col1,new_col2,new_col3,new_col4,new_col5)
#         except:
#             print('Polynomial-Interaction: something is wrong')

# Interaction features
print('Interaction features: ')

for i in range(len(X.columns)):
    if X.columns[i] not in poly_cols:
        for j in range(i+1,len(X.columns)):
            if X.columns[j] not in poly_cols:
                col1=X.columns[i]
                col2=X.columns[j]
                new_col=col1+'_'+col2
                pvalue=stats.pearsonr(X[col1], X[col2])[1] # Check for p-value
                if pvalue <=0.00000000000000000000000000000000000000000000000000000000000000000000000000000000000000001:
                    X[new_col]=X[col1]*X[col2]
                    interaction_cols.append(new_col)
                    print(col1,' and ',col2,' p-value is: ',pvalue)

# Rank features
print('Rank features: ')
for col in X.columns:
    if col not in newColumnsNom and col not in columnsOrd:
        new_col='rank_'+col
        X[new_col]=X[col].rank()
        rank_cols.append(new_col)
        print(new_col)
        
# Log features        
print('Log features: ')
for col in X.columns:
    if col not in newColumnsNom:
        new_col='log_'+col
        min_val=min(X[col])
        shift_by=abs(min_val)+1 # Shift by 1 to prevent log(0)
        X[new_col]=X[col]+shift_by
        X[new_col]=np.log(X[new_col])
        log_cols.append(new_col)
        print(new_col)
        
# Bucketing/binning/categorization
print('Binning features: ')
for col in X.columns:
    if col not in newColumnsNom and col not in interaction_cols and col not in rank_cols:
        try:
            new_col='bin_'+col
            min_val=min(X[col])
            max_val=max(X[col])
            bins=round(abs(max_val-min_val)/X[col].size)
            if bins>2:
                X[new_col]=pd.cut(X[col], bins,labels=range(bins))
                X[[new_col]]=X[[new_col]].astype(float)
                binning_cols.append(new_col)
                print(new_col)
        except:
            print('Binning: something is wrong')

Polynomial features: 
poly_LotFrontage
poly_LotArea
poly_LotShape
poly_Utilities
poly_LandSlope
poly_OverallQual
poly_OverallCond
poly_YearBuilt
poly_YearRemodAdd
poly_MasVnrArea
poly_ExterQual
poly_ExterCond
poly_BsmtQual
poly_BsmtCond
poly_BsmtExposure
poly_BsmtFinType1
poly_BsmtFinSF1
poly_BsmtFinType2
poly_BsmtFinSF2
poly_BsmtUnfSF
poly_TotalBsmtSF
poly_HeatingQC
poly_CentralAir
poly_1stFlrSF
poly_2ndFlrSF
poly_LowQualFinSF
poly_GrLivArea
poly_BsmtFullBath
poly_BsmtHalfBath
poly_FullBath
poly_HalfBath
poly_BedroomAbvGr
poly_KitchenAbvGr
poly_KitchenQual
poly_TotRmsAbvGrd
poly_Functional
poly_Fireplaces
poly_FireplaceQu
poly_GarageYrBlt
poly_GarageFinish
poly_GarageCars
poly_GarageArea
poly_GarageQual
poly_GarageCond
poly_PavedDrive
poly_WoodDeckSF
poly_OpenPorchSF
poly_EnclosedPorch
poly_3SsnPorch
poly_ScreenPorch
poly_PoolArea
poly_PoolQC
poly_Fence
poly_MiscVal
poly_MoSold
poly_YrSold
Interaction features: 
LandSlope  and  Nominal_LandContour_Low  p-value is:  3.045934574042485e-

1stFlrSF  and  GrLivArea  p-value is:  1.936812537028707e-124
1stFlrSF  and  OverallQual_TotalBsmtSF  p-value is:  0.0
1stFlrSF  and  OverallQual_GrLivArea  p-value is:  7.104464167641514e-141
1stFlrSF  and  OverallQual_GarageCars  p-value is:  4.708860613682712e-103
1stFlrSF  and  OverallQual_GarageArea  p-value is:  1.0957184579469062e-125
1stFlrSF  and  YearBuilt_OverallQual_GarageCars  p-value is:  8.030114175427934e-103
1stFlrSF  and  YearBuilt_OverallQual_GarageArea  p-value is:  1.8226390493552246e-125
1stFlrSF  and  YearRemodAdd_OverallQual_GarageCars  p-value is:  7.308619857575803e-103
1stFlrSF  and  YearRemodAdd_YearBuilt_OverallQual_GarageCars  p-value is:  1.3669391107113859e-102
1stFlrSF  and  YearRemodAdd_YearBuilt_OverallQual_GarageArea  p-value is:  3.7609598851628437e-125
1stFlrSF  and  BsmtFinSF1_BsmtUnfSF  p-value is:  4.4656480053890145e-132
1stFlrSF  and  BsmtFinSF1_TotalBsmtSF  p-value is:  1.2298501946668916e-110
1stFlrSF  and  BsmtFinSF1_OverallQual_TotalBsmtSF

GrLivArea  and  2ndFlrSF_BedroomAbvGr  p-value is:  1.047588749386586e-227
GrLivArea  and  2ndFlrSF_TotRmsAbvGrd  p-value is:  8.707355911037427e-265
GrLivArea  and  2ndFlrSF_Nominal_MSSubClass_60  p-value is:  3.6037146689798354e-118
GrLivArea  and  2ndFlrSF_Nominal_HouseStyle_2Story  p-value is:  7.078493191570732e-135
GrLivArea  and  2ndFlrSF_OverallQual_GrLivArea  p-value is:  3.06110333835054e-310
BsmtFullBath  and  BsmtFinSF1_BsmtFullBath  p-value is:  1.0503548522583719e-290
FullBath  and  TotRmsAbvGrd  p-value is:  1.2901193443555294e-118
FullBath  and  OverallQual_YearBuilt  p-value is:  1.260244751386697e-121
FullBath  and  OverallQual_YearRemodAdd  p-value is:  1.6222934347186475e-119
FullBath  and  OverallQual_GrLivArea  p-value is:  5.043159042605312e-162
FullBath  and  OverallQual_FullBath  p-value is:  0.0
FullBath  and  OverallQual_GarageYrBlt  p-value is:  4.730693020461962e-121
FullBath  and  OverallQual_GarageCars  p-value is:  2.347382366690964e-117
FullBath  and  O

KitchenQual  and  ExterQual_KitchenQual  p-value is:  0.0
TotRmsAbvGrd  and  OverallQual_GrLivArea  p-value is:  8.177588065261356e-233
TotRmsAbvGrd  and  OverallQual_FullBath  p-value is:  3.107590507557984e-123
TotRmsAbvGrd  and  YearBuilt_OverallQual_FullBath  p-value is:  2.499712699660314e-119
TotRmsAbvGrd  and  YearRemodAdd_OverallQual_FullBath  p-value is:  8.327506775192685e-122
TotRmsAbvGrd  and  YearRemodAdd_YearBuilt_OverallQual_FullBath  p-value is:  5.970718603919241e-118
TotRmsAbvGrd  and  1stFlrSF_GrLivArea  p-value is:  4.994146631464527e-153
TotRmsAbvGrd  and  1stFlrSF_OverallQual_GrLivArea  p-value is:  7.145313053404873e-117
TotRmsAbvGrd  and  2ndFlrSF_GrLivArea  p-value is:  1.4652314668389675e-188
TotRmsAbvGrd  and  2ndFlrSF_HalfBath  p-value is:  1.4493155339998092e-91
TotRmsAbvGrd  and  2ndFlrSF_BedroomAbvGr  p-value is:  8.779024493006087e-191
TotRmsAbvGrd  and  2ndFlrSF_TotRmsAbvGrd  p-value is:  3.5166999925511205e-228
TotRmsAbvGrd  and  2ndFlrSF_Nominal_House

Fireplaces  and  FireplaceQu  p-value is:  8.690104375245839e-238
FireplaceQu  and  Fireplaces_FireplaceQu  p-value is:  2.2296327633406404e-116
GarageYrBlt  and  GarageCars  p-value is:  9.808732069854972e-156
GarageYrBlt  and  GarageArea  p-value is:  3.7338244232195626e-145
GarageYrBlt  and  Nominal_Exterior1st_VinylSd  p-value is:  1.2002405987844281e-95
GarageYrBlt  and  Nominal_Exterior2nd_VinylSd  p-value is:  2.501022549716943e-93
GarageYrBlt  and  Nominal_Foundation_PConc  p-value is:  1.6657255034551967e-167
GarageYrBlt  and  OverallQual_YearBuilt  p-value is:  5.913280652073305e-137
GarageYrBlt  and  OverallQual_YearRemodAdd  p-value is:  1.0366855641552875e-125
GarageYrBlt  and  OverallQual_FullBath  p-value is:  8.966055956016244e-126
GarageYrBlt  and  OverallQual_GarageYrBlt  p-value is:  4.956204763099412e-139
GarageYrBlt  and  OverallQual_GarageCars  p-value is:  1.117942108220541e-177
GarageYrBlt  and  OverallQual_GarageArea  p-value is:  5.113502536936853e-164
GarageY

...

log_rank_Nominal_Foundation_PConc_Nominal_Exterior1st_VinylSd_GarageArea_GarageCars_GarageYrBlt_YearRemodAdd_YearBuilt_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior1st_VinylSd_GarageArea_GarageCars_GarageYrBlt_HeatingQC_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior1st_VinylSd_GarageArea_GarageCars_GarageYrBlt_HeatingQC_YearBuilt_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior1st_VinylSd_GarageArea_GarageCars_GarageYrBlt_HeatingQC_YearRemodAdd_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior1st_VinylSd_GarageArea_GarageCars_GarageYrBlt_HeatingQC_YearRemodAdd_YearBuilt_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nomina

log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_GarageCars_GarageYrBlt_HeatingQC_YearRemodAdd_YearBuilt_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_GarageCars_GarageYrBlt_FullBath_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_GarageCars_GarageYrBlt_FullBath_YearBuilt_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_GarageCars_GarageYrBlt_FullBath_YearRemodAdd_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_GarageCars_GarageYrBlt_FullBath_YearRemodAdd_YearBuilt_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_GarageCars_GarageYrBlt_FullBath_HeatingQC_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_GarageCars_GarageYrBlt_FullBath_HeatingQC_YearBuilt_OverallQual_Nominal_

log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_HeatingQC_YearRemodAdd_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_HeatingQC_YearRemodAdd_YearBuilt_YearRemodAdd
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_HeatingQC_YearRemodAdd_YearBuilt_GarageYrBlt
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_HeatingQC_YearRemodAdd_YearBuilt_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_HeatingQC_YearRemodAdd_YearBuilt_OverallQual_YearRemodAdd
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_HeatingQC_YearRemodAdd_YearBuilt_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_FullBath_OverallQual_Nominal_Foun

log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_GarageCars_GarageYrBlt_HeatingQC_YearRemodAdd_YearBuilt_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_GarageCars_GarageYrBlt_FullBath_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_GarageCars_GarageYrBlt_FullBath_YearBuilt_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_GarageCars_GarageYrBlt_FullBath_YearRemodAdd_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_GarageCars_GarageYrBlt_FullBath_YearRemodAdd_YearBuilt_OverallQual_Nominal_Foundation_PConc
log_rank_Nominal_Foundation_PConc_Nominal_Exterior2nd_VinylSd_Nominal_Exterior1st_VinylSd_GarageCars_GarageYrBlt_FullBath_HeatingQC_Ove

log_rank_Nominal_SaleType_New_Nominal_SaleCondition_Partial
log_rank_Nominal_SaleType_WD_Nominal_SaleCondition_Normal
log_rank_Nominal_SaleType_WD_Nominal_SaleCondition_Partial
log_rank_Nominal_SaleType_WD_Nominal_SaleType_New_Nominal_SaleCondition_Partial
log_rank_Nominal_SaleCondition_Abnorml_Nominal_SaleCondition_Normal
log_rank_Nominal_SaleCondition_Abnorml_Nominal_SaleType_WD_Nominal_SaleCondition_Normal
log_rank_Nominal_SaleCondition_Normal_Nominal_SaleCondition_Partial
log_rank_Nominal_SaleCondition_Normal_Nominal_SaleType_New_Nominal_SaleCondition_Partial
log_rank_Nominal_SaleCondition_Normal_Nominal_SaleType_WD_Nominal_SaleCondition_Normal
log_rank_Nominal_SaleCondition_Partial_Nominal_SaleType_New_Nominal_SaleCondition_Partial
log_rank_Nominal_SaleCondition_Partial_Nominal_SaleType_WD_Nominal_SaleCondition_Normal
log_rank_Nominal_SaleCondition_Partial_Nominal_SaleCondition_Normal_Nominal_SaleType_WD_Nominal_SaleCondition_Normal
log_rank_Nominal_Street_Grvl_Nominal_Street_Pave

### Remove duplicates and constant-valued columns

In [12]:
def getDuplicateColumns(df): 
  
    duplicateColumnNames = set() 
      
    for x in range(df.shape[1]): 
        col = df.iloc[:, x] 
        for y in range(x + 1, df.shape[1]): 
            otherCol = df.iloc[:, y] 
            if col.equals(otherCol): 
                duplicateColumnNames.add(df.columns.values[y]) 
    return list(duplicateColumnNames) 

X = X.drop(columns = getDuplicateColumns(df)) 
X.shape

(1460, 15664)

In [13]:
from sklearn.feature_selection import VarianceThreshold

def variance_threshold_selector(data, threshold=0.5):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

X=pd.DataFrame(X)
X = variance_threshold_selector(X,0.0)

### Scaling

In [14]:
# Select k-best columns (optional)
from sklearn.feature_selection import SelectKBest, chi2

# X = SelectKBest(chi2, k=4).fit_transform(X, y)
X.shape

(1460, 14076)

In [15]:
# Scaling for small sets
from sklearn import preprocessing

# X=preprocessing.scale(X)
# X.shape

In [16]:
X=pd.DataFrame(X)
X= pd.concat([X, y], axis=1, join='inner')
names = X.columns
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(X)
X = pd.DataFrame(scaled_df, columns=names)
y=X['SalePrice']
X=X.drop(['SalePrice'], axis=1)

## Modeling

In [17]:
# Remove columns with Nans if created
noNanCols=[]
for col in X.columns:
    if np.any(np.isnan(X[col])):
        print(col)
    else:
        noNanCols.append(col)

X=X[noNanCols]

log_GrLivArea_1stFlrSF_TotalBsmtSF_YearRemodAdd_YearBuilt_OverallQual_YearBuilt
log_GrLivArea_1stFlrSF_TotalBsmtSF_YearRemodAdd_YearBuilt_OverallQual_YearRemodAdd
log_GrLivArea_1stFlrSF_TotalBsmtSF_YearRemodAdd_YearBuilt_OverallQual_GarageCars
log_GarageArea_GrLivArea_1stFlrSF_YearRemodAdd_YearBuilt_OverallQual_GarageArea
log_GarageArea_TotRmsAbvGrd_GrLivArea_HeatingQC_YearRemodAdd_YearBuilt_OverallQual_YearRemodAdd
log_GarageArea_TotRmsAbvGrd_GrLivArea_1stFlrSF_YearRemodAdd_YearBuilt_OverallQual_GarageCars
log_GarageArea_TotRmsAbvGrd_FullBath_GrLivArea_YearRemodAdd_YearBuilt_OverallQual_GarageArea
log_GarageArea_GarageCars_FullBath_GrLivArea_HeatingQC_YearRemodAdd_YearBuilt_OverallQual_YearRemodAdd
log_GarageArea_GarageCars_TotRmsAbvGrd_GrLivArea_YearRemodAdd_YearBuilt_OverallQual_YearBuilt
log_GarageArea_GarageCars_TotRmsAbvGrd_GrLivArea_YearRemodAdd_YearBuilt_OverallQual_YearRemodAdd
log_GarageArea_GarageCars_TotRmsAbvGrd_GrLivArea_YearRemodAdd_YearBuilt_OverallQual_GarageArea
log_G

### Dataset splitting, Model training, Cross-validation

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.datasets import make_regression
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import metrics
from sklearn.model_selection import cross_val_score
    
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print('Training in progress...')

clf = GradientBoostingRegressor(random_state=0)
scores = cross_val_score(clf, X, y, cv=10)
cv_score=sum(scores)/len(scores)
cv_score

Training in progress...


0.8972659490221501

### Learning curves

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

print('Printing...')

def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):

    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

fig, axes = plt.subplots(3, 2, figsize=(10, 15))

title = "Learning Curves"

cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = clf
plot_learning_curve(estimator, title, X, y, axes=axes[:, 0], ylim=(0.2, 1.01),
                    cv=cv, n_jobs=4)

plt.show()

Printing...


### Scores

| Model | Accuracy   |
|------|------|
| Lasso | 0.83 |
| ElasticNet | 0.55 |
| Ridge | 0.26 |
| SVR(kernel='linear')| 0.69 |
| SVR(kernel='rbf') | 0.86 |
| BaggingRegressor | 0.56 |
| RidgeCV | <0.24 |
| RandomForestRegressor | 0.87 |
| AdaBoostRegressor | 0.81 |
| GradientBoostingRegressor | 0.90 |

## Conclusion
After taking into account underfitting and overfitting, the best model to solve this problem is GradientBoostingRegressor.

### Future development of the project
Extend a dataset with features created using splines method.

Take advantage of cloud computing to choose best possible dataset and model.
